In [4]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

# Paths
input_path = r"C:\Users\black\Downloads\microdatos_manzana\Microdatos_Manzana.shp"
out_dir = Path(r"C:\Users\black\Downloads\microdatos_manzana\Centroide")
out_dir.mkdir(parents=True, exist_ok=True)

# Load
gdf = gpd.read_file(input_path)

# Regiones para EPSG 32718
regiones_epsg32718 = [
    "REGIÓN DE ATACAMA",
    "REGIÓN DE ARICA Y PARINACOTA",
    "REGIÓN DE COQUIMBO",
    "REGIÓN DE VALPARAÍSO",
    "REGIÓN DE TARAPACÁ",
    "REGIÓN DE ANTOFAGASTA"
]

# Calcular centroides
gdf["geometry"] = gdf.geometry.centroid

# Dividir y reproyectar
gdf_18 = gdf[gdf["REGION"].isin(regiones_epsg32718)].to_crs(epsg=32718)[["CUT", "TOTAL_PERS", "geometry"]]
gdf_19 = gdf[~gdf["REGION"].isin(regiones_epsg32718)].to_crs(epsg=32719)[["CUT", "TOTAL_PERS", "geometry"]]

# Guardar shapefiles separados
out18 = out_dir / "Centroides_Manzana_32718.shp"
out19 = out_dir / "Centroides_Manzana_32719.shp"

gdf_18.to_file(out18, driver="ESRI Shapefile", encoding="utf-8")
gdf_19.to_file(out19, driver="ESRI Shapefile", encoding="utf-8")

print(f"Exportado: {out18}")
print(f"Exportado: {out19}")



Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Centroides_Manzana_32718.shp
Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Centroides_Manzana_32719.shp


In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path

# === Paths ===
poly_path = r"C:\Users\black\Downloads\microdatos_entidad\Microdatos_Entidad.shp"
points_path = r"C:\Users\black\Downloads\viviendas-rurales-precenso-2016\Viviendas_Rurales_Precenso_2016.shp"
out_dir = Path(r"C:\Users\black\Downloads\microdatos_manzana\Centroide")
out_dir.mkdir(parents=True, exist_ok=True)

# === Leer shapefiles ===
polys = gpd.read_file(poly_path)
pts = gpd.read_file(points_path)

# === Regiones EPSG 32718 ===
regiones_32718 = [1, 2, 3, 4, 5, 15]

# Asegurar que ambos tengan el mismo CRS para hacer spatial join
if polys.crs != pts.crs:
    pts = pts.to_crs(polys.crs)

# === Join espacial: asignar puntos a polígonos ===
joined = gpd.sjoin(pts, polys, how="inner", predicate="within")

# === Calcular promedio de coordenadas por polígono ===
means = (
    joined.groupby("index_right")
    .apply(lambda g: pd.Series({
        "x_mean": g.geometry.x.mean(),
        "y_mean": g.geometry.y.mean()
    }))
)

# === Unir resultados con atributos de los polígonos ===
polys_reset = polys.reset_index()
result = polys_reset.merge(means, left_index=True, right_index=True)

# Crear geometría con el promedio
result["geometry"] = [Point(xy) for xy in zip(result["x_mean"], result["y_mean"])]

# Seleccionar columnas necesarias
result = result[["COD_COMUNA", "TOTAL_PERS", "COD_REGION", "geometry"]]
result = gpd.GeoDataFrame(result, geometry="geometry", crs=polys.crs)

# === Separar por EPSG ===
gdf_18 = result[result["COD_REGION"].astype(int).isin(regiones_32718)].to_crs(epsg=32718)
gdf_19 = result[~result["COD_REGION"].astype(int).isin(regiones_32718)].to_crs(epsg=32719)

# === Guardar ===
out18 = out_dir / "Entidad_Promedios_32718.shp"
out19 = out_dir / "Entidad_Promedios_32719.shp"

gdf_18.to_file(out18, driver="ESRI Shapefile", encoding="utf-8")
gdf_19.to_file(out19, driver="ESRI Shapefile", encoding="utf-8")

print(f"Exportado: {out18}")
print(f"Exportado: {out19}")


C:\Users\black\AppData\Local\Temp\ipykernel_24588\1824467658.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Entidad_Promedios_32718.shp
Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Entidad_Promedios_32719.shp


In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path

# === Paths ===
excel_path = r"C:\Users\black\Dropbox\Proyectos\stations_info_stata.xlsx"
out_dir = Path(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide")
out_dir.mkdir(parents=True, exist_ok=True)

# === Leer Excel ===
df = pd.read_excel(excel_path)

# Eliminar duplicados
df = df.drop_duplicates(subset=["Estación"])

# Separar coordenadas UTM: "282763 E 6848691 N"
df[["x", "dummyE", "y", "dummyN"]] = df["CoordenadasUTM"].str.split(expand=True)
df["x"] = df["x"].astype(float)
df["y"] = df["y"].astype(float)

# Crear geometría
geometry = [Point(xy) for xy in zip(df["x"], df["y"])]
df["geometry"] = geometry

# Convertir a GeoDataFrame sin CRS inicial
gdf = gpd.GeoDataFrame(df, geometry="geometry")

# Convertir fechas a string (para shapefile)
for col in ["date_inicio", "date_fin"]:
    if col in gdf.columns:
        gdf[col] = gdf[col].astype(str)

# === Separar por HUSO ===
gdf_18 = gdf[gdf["Huso"] == 18].set_crs(epsg=32718, allow_override=True)
gdf_19 = gdf[gdf["Huso"] == 19].set_crs(epsg=32719, allow_override=True)

# === Exportar shapefiles ===
out18 = out_dir / "Coordenadas_Huso18.shp"
out19 = out_dir / "Coordenadas_Huso19.shp"

if not gdf_18.empty:
    gdf_18.to_file(out18, driver="ESRI Shapefile", encoding="utf-8")
    print(f"Exportado: {out18}")

if not gdf_19.empty:
    gdf_19.to_file(out19, driver="ESRI Shapefile", encoding="utf-8")
    print(f"Exportado: {out19}")


C:\Users\black\AppData\Local\Temp\ipykernel_2396\2409089919.py:43: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_18.to_file(out18, driver="ESRI Shapefile", encoding="utf-8")


Exportado: C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Coordenadas_Huso18.shp
Exportado: C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Coordenadas_Huso19.shp


C:\Users\black\AppData\Local\Temp\ipykernel_2396\2409089919.py:47: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_19.to_file(out19, driver="ESRI Shapefile", encoding="utf-8")
